In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#import, change and sort data
prices = pd.read_csv('/prices.csv')
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
prices.sort_values('updated_at', inplace = True)
print(prices.head())

    product_id  old_price  new_price          updated_at
4      3954203      68800      60000 2018-09-10 16:32:00
7      3998909      19000      17000 2018-09-10 16:35:00
0           64     270000     239000 2018-09-10 16:37:00
11     4085861      60000      53500 2018-09-11 08:51:00
1      3954203      60000      64000 2018-09-11 11:54:00


In [ ]:
#import, change and sort data
sales = pd.read_csv('/sales.csv')
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
sales.sort_values('ordered_at', inplace = True)
print(sales.head())

     product_id          ordered_at  quantity_ordered
86      3954203 2018-09-11 01:43:00                 1
28      4085861 2018-09-11 06:26:00                 1
26      4085861 2018-09-11 06:53:00                 1
27      4085861 2018-09-11 08:24:00                 1
123     4085861 2018-09-11 09:30:00                 1


In [ ]:
#merge 2 tables and find correct price
new_sales = pd.merge_asof(sales, prices,
                          left_on='ordered_at',
                          right_on='updated_at',
                          by='product_id',
                          direction='nearest')
df1 = new_sales.loc[new_sales['ordered_at'] >= new_sales['updated_at'], ['product_id', 'new_price', 'quantity_ordered']]
df2 = new_sales.loc[new_sales['ordered_at'] <  new_sales['updated_at'], ['product_id', 'old_price', 'quantity_ordered']]
df2['new_price'] = df2['old_price']
df2= df2[['product_id','quantity_ordered','new_price']]

In [ ]:
#create final data
data = pd.concat([df1,df2])
data = data.rename(columns={'new_price':'final_price'})
final_data = data.groupby(['product_id','final_price'], as_index= False).sum('quantity_ordered')
print(final_data)

    product_id  final_price  quantity_ordered
0           64       239000                 4
1      3954203        57500                 1
2      3954203        60000                 3
3      3954203        64000                10
4      3998909        15500                 1
5      3998909        16500                14
6      3998909        17000                 2
7      4085861        52000                20
8      4085861        53500                40
9      4085861        58000                38
10     4085861        60000                 3
11     4085861        62500                29
12     4085861        67000                13


In [ ]:
#revenue of each product at each price
final_data['revenue'] = final_data['final_price'] * final_data['quantity_ordered']
final_data = final_data[['product_id','final_price','revenue']]
print(final_data)

    product_id  final_price  revenue
0           64       239000   956000
1      3954203        57500    57500
2      3954203        60000   180000
3      3954203        64000   640000
4      3998909        15500    15500
5      3998909        16500   231000
6      3998909        17000    34000
7      4085861        52000  1040000
8      4085861        53500  2140000
9      4085861        58000  2204000
10     4085861        60000   180000
11     4085861        62500  1812500
12     4085861        67000   871000


In [ ]:
#revenue of each product
fn1 = final_data.groupby('product_id', as_index = False).sum('revenue')
print(fn1)

   product_id  final_price  revenue
0          64       239000   956000
1     3954203       181500   877500
2     3998909        49000   280500
3     4085861       353000  8247500
